In [2]:
import $file.hw6stdlib
import hw6stdlib._

<console>: 89

defined trait Nat
defined object Zero
defined class Succ
one: Succ = Succ(Zero)
two: Succ = Succ(Succ(Zero))
three: Succ = Succ(Succ(Succ(Zero)))
four: Succ = Succ(Succ(Succ(Succ(Zero))))
five: Succ = Succ(Succ(Succ(Succ(Succ(Zero)))))
six: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))
seven: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))
eight: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))))
nine: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))))
ten: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))))))
nat_plus: (x: Nat, y: Nat)Nat
nat_to_int: (x: Nat)Int
nat_to_str: (x: Nat)String
print_nat: (x: Nat)Unit
nat_mult: (x: Nat, y: Nat)Nat
nat_minus: (x: Nat, y: Nat)Nat
nat_pow: (x: Nat, y: Nat)Nat
nat_lte: (x: Nat, y: Nat)Bool
define...

# Homework 6

Due 10/20 at 11:59pm

In this assignment we will write the first full interpreter for Lettuce. Remember that lettuce is a functional language with let bindings. It is very similiar to a language called ML. Here we want to implement the interpreter for the language from the ground up. We will use most of the existing standard library we have developed while we write this, especially the equality functions and Dictionary data type from the last homework.

## Submission Instructions
Upload only this .ipynb file to Canvas. Do not add anything to stdlib since you can't submit it.

In this homework we will define integers and their operations, then build an interpreter.

See [this link](https://www.notion.so/Guidelines-for-Programming-Homework-dbd25efa7bb24915ae6bcb06827fc5b6) for what is and isn't allowed in your code. 



## Problem 1 - The Values (5 points)

Recall that we define interpreters as functions that take in expressions in the form of abstract syntax and give a value as output. In symbols:

$$
eval : Expr \rightarrow Value
$$

It follows that we will need to create a datatype to represent the values that may be computed by lettuce. Bellow is the grammar that defines lettuce values. Implemenet this as a `sealead trait` in Scala below:

$$
\begin{align}
\textbf{Value} ::=&\ NumVal\ \mathbb{Z}\\
\mid&\ BinVal\ \mathbb{B}\\
\mid&\ Error
\end{align}
$$

In [2]:
sealed trait Value
case object Error extends Value
case class NumVal(x:Integer) extends Value
case class BinVal(x:Bool) extends Value

defined trait Value
defined object Error
defined class NumVal
defined class BinVal
Companions must be defined together; you may wish to use :paste mode for this.


If your definition was correct then the values in the cell below should compile.

In [3]:
val v1 = NumVal(Positive(Succ(Succ(Zero))))
val v2 = BinVal(False)
val v3 = Error
passed(5)


*** Tests Passed (5 points) ***


null

## Problem 2 - The Expressions (10 points)

There are many possible expressions in the Lettuce language. We also use an abstract syntax to represent this. Here is the grammar to remind you of all the syntactic elements in Lettuce:

$$\begin{array}{rcll}
\mathbf{Expr} & ::= & Const(\mathbb{Z}) \\
 & | & Bin(\mathbb{B}) \\
 & | & Ident(\mathbf{String}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Pow (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Neg (\mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{String}, \mathbf{Expr}, \mathbf{Expr}) \\
\end{array}$$

Now define a `sealed trait` for expressions in Lettuce:

In [4]:
sealed trait Expr
case class Const(x:Integer) extends Expr
case class Bin(x:Bool) extends Expr
case class Ident(x:String) extends Expr
case class Plus(x:Expr, y:Expr) extends Expr
case class Minus(x:Expr, y:Expr) extends Expr
case class Mult(x:Expr, y:Expr) extends Expr
case class Pow(x:Expr, y:Expr) extends Expr
case class Neg(x:Expr) extends Expr
case class Eq(x:Expr, y:Expr) extends Expr
case class And(x:Expr, y:Expr) extends Expr
case class Or(x:Expr, y:Expr) extends Expr
case class IfThenElse(x:Expr, y:Expr, z:Expr)extends Expr
case class Let(x:String, y:Expr, z:Expr) extends Expr

defined trait Expr
defined class Const
defined class Bin
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Let


If you defined `Expr` properly then the following expressions should compile:

In [5]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
passed(5)


*** Tests Passed (5 points) ***


null

In [6]:
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))
passed(5)


*** Tests Passed (5 points) ***


null

## Problem 3 - The Interpreter (30 pts)

We now need to define the interpreter for this language. It is a function of the form:

$$
eval : Expr \rightarrow Value
$$

Except now that we have an execution environment($\sigma$) we need some way to include sigma in this function. This will be as an additional argument:

$$
eval : \sigma \rightarrow Expr \rightarrow Value
$$

We will let $\sigma$ be a `Dictionary` as we defined in last week's homework. To add something to a dicitonary use the `Entry` constructor to add on a new element. This should be very similiar to the way `Cons` works on lists.

Below we have given you the skeleton of the function `eval`. Fill in each case for the interpreter so that it can interpret all possible Lettuce expressions. The inference rules we covered in the classroom should be very helpful with this.

Hint: It may be helpful to define some helper functions for the inference rules that have similar forms. These are the rules like binary operations for arithmetic, bool, etc. Just like we lumped some of these rules together into a single rule, we can do the same for our interpreter in the form of auxillary functions. The bonus `eval_bin` function from Homework 4 may be a good place for inspiration.

Hint: Be ready to write some nested case matches. Most of your cases should have two nestings. One for the Sytnax element and another for evaluating the expressions it may contain. This is not true for all expressions but most will 

In [12]:
def eval_bin(f : (Integer, Integer) => Integer, e1 : Expr, e2 : Expr, env: Dictionary[String, Value]) : Value = (e1, e2) match {
    case (x, y) => (eval(env, x), eval(env, y)) match {
        case (NumVal(x) , NumVal(y)) => NumVal(f(x, y))
    }
}

def eval_bin_AndOr(f: (Bool, Bool) => Bool, e1: Expr, e2: Expr, env: Dictionary[String, Value]) : Value = (e1, e2) match {
     case (x, y) => (eval(env, x), eval(env, y)) match {
        case (BinVal(x) , BinVal(y)) => BinVal(f(x, y))
    }
}

def eval_bin_bool(f: (Integer, Integer) => Bool, e1: Expr, e2: Expr, env: Dictionary[String, Value]) : Value = (e1, e2) match {
     case (x, y) => (eval(env, x), eval(env, y)) match {
        case (NumVal(x) , NumVal(y)) => BinVal(f(x, y))
    }
}

def eval(env : Dictionary[String, Value], expr : Expr) : Value = expr match {
    case Const(x) => NumVal(x)
    case Bin(x) => BinVal(x)
    case Ident(x) => lookup(string_eq, env, x) match {
        case Just(x) => x
        case None => Error
    }
    case Plus(x, y) => eval_bin(plus, x, y, env) 
    case Minus(x, y) => eval_bin(minus, x, y, env)
    case Mult(x, y) => eval_bin(mult, x, y, env)
    case Pow(x, y) => (eval(env, x), eval(env, y)) match {
        case(NumVal(x), NumVal(y)) => NumVal(pow(x, abs(y)))
    }
    case Neg(x) => eval(env, x) match {
        case(NumVal(x)) => NumVal(negate(x))
    }
        case IfThenElse(p, e1, e2) => eval(env, p) match {
            case BinVal(True) => eval(env, e1)
            case BinVal(False) => eval(env, e2)
        }       
    case Let(s, e1, e2) => eval(Entry(s, eval(env, e1), EmptyDict), e2) 
    
    case Eq(x, y) => eval_bin_bool(int_eq, x, y, env)
    case And(x, y) => eval_bin_AndOr(and, x, y, env)
    case Or(x, y) => eval_bin_AndOr(or, x, y, env)
    

}

<console>:153: warning: match may not be exhaustive.
It would fail on the following inputs: (BinVal(_), BinVal(_)), (BinVal(_), Error), (BinVal(_), NumVal(_)), (Error, BinVal(_)), (Error, Error), (Error, NumVal(_)), (NumVal(_), BinVal(_)), (NumVal(_), Error)
       case (x, y) => (eval(env, x), eval(env, y)) match {
                      ^
<console>:159: warning: match may not be exhaustive.
It would fail on the following inputs: (BinVal(_), Error), (BinVal(_), NumVal(_)), (Error, BinVal(_)), (Error, Error), (Error, NumVal(_)), (NumVal(_), BinVal(_)), (NumVal(_), Error), (NumVal(_), NumVal(_))
       case (x, y) => (eval(env, x), eval(env, y)) match {
                      ^
<console>:165: warning: match may not be exhaustive.
It would fail on the following inputs: (BinVal(_), BinVal(_)), (BinVal(_), Error), (BinVal(_), NumVal(_)), (Error, BinVal(_)), (Error, Error), (Error, NumVal(_)), (NumVal(_), BinVal(_)), (NumVal(_), Error)
       case (x, y) => (eval(env, x), eval(env, y)) match 

If this was defined correctly then the definitions above should evaluate:

In [8]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))

Let(x,Plus(Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))),Const(Negative(Succ(Succ(Succ(Zero)))))),Mult(Ident(x),Ident(x)))

In [9]:
assert(eval(EmptyDict, ex3) == BinVal(True))
passed(10)


*** Tests Passed (10 points) ***


In [10]:
assert(eval(EmptyDict, ex4) == Error)
passed(10)


*** Tests Passed (10 points) ***


In [11]:
assert(eval(EmptyDict, ex5) == NumVal(Positive(two)))
assert(eval(EmptyDict, ex6) == NumVal(Positive(four)))
passed(10)


*** Tests Passed (10 points) ***
